In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.callbacks import EarlyStopping

In [ ]:
size = [224, 224]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/train'
test_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/test'

In [ ]:
resnet = ResNet50(input_shape=size + [3], weights='imagenet', include_top=False)


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
data = glob('/content/drive/MyDrive/HDA2/Covid19-dataset/train/*')

In [ ]:
x = Flatten()(resnet.output)
prediction = Dense(len(data), activation='softmax')(x)

In [ ]:
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_img = ImageDataGenerator(rescale = 1./255,shear_range = 0.25,zoom_range = 0.3)
test_img = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/train',target_size=size)

Found 251 images belonging to 3 classes.


In [ ]:
for i in range(5):
    batch = next(training_set)
    data_point = batch[0][0]  # First image in the batch
    label = batch[1][0]  # Corresponding label

    print(f"Data Point {i + 1} - Label: {label}")

Data Point 1 - Label: [1. 0. 0.]
Data Point 2 - Label: [1. 0. 0.]
Data Point 3 - Label: [1. 0. 0.]
Data Point 4 - Label: [0. 1. 0.]
Data Point 5 - Label: [0. 0. 1.]


In [ ]:
testing_set = test_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

r = model.fit(
  training_set,
  validation_data=testing_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_set),
  callbacks=[early_stop]
)

Epoch 1/50
8/8 [==============================] - 64s 7s/step - loss: 7.6451 - accuracy: 0.3426 - val_loss: 7.1561 - val_accuracy: 0.3030
Epoch 2/50
8/8 [==============================] - 62s 8s/step - loss: 4.0799 - accuracy: 0.3944 - val_loss: 4.1010 - val_accuracy: 0.6212
Epoch 3/50
8/8 [==============================] - 71s 9s/step - loss: 3.0941 - accuracy: 0.5498 - val_loss: 3.4713 - val_accuracy: 0.3485
Epoch 4/50
8/8 [==============================] - 60s 7s/step - loss: 1.9861 - accuracy: 0.5817 - val_loss: 0.7135 - val_accuracy: 0.6818
Epoch 5/50
8/8 [==============================] - 70s 9s/step - loss: 1.2842 - accuracy: 0.5896 - val_loss: 0.4737 - val_accuracy: 0.8030
Epoch 6/50
8/8 [==============================] - 70s 9s/step - loss: 1.1337 - accuracy: 0.6335 - val_loss: 1.7765 - val_accuracy: 0.6061
Epoch 7/50
8/8 [==============================] - 61s 7s/step - loss: 1.0799 - accuracy: 0.6454 - val_loss: 0.5339 - val_accuracy: 0.8182
Epoch 8/50
8/8 [==================

In [ ]:
model.save('Lung_disease_pred_resnet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np


model_resnet = load_model('/content/drive/MyDrive/HDA2/Lung_disease_pred_resnet.h5')

In [ ]:

y_pred_resnet = model_resnet.predict(testing_set)


y_pred_classes_resnet = np.argmax(y_pred_resnet, axis=1)
y_true_classes = testing_set.classes

accuracy_resnet = accuracy_score(y_true_classes, y_pred_classes_resnet)
print("Accuracy (Resnet):", accuracy_resnet)


print("Classification Report (Resnet):")
print(classification_report(y_true_classes, y_pred_classes_resnet))


conf_matrix_resnet = confusion_matrix(y_true_classes, y_pred_classes_resnet)
print("Confusion Matrix (Resnet):")
print(conf_matrix_resnet)

9/9 [==============================] - 14s 1s/step
Accuracy (Resnet): 0.2878787878787879
Classification Report (Resnet):
              precision    recall  f1-score   support

           0       0.32      0.35      0.33        26
           1       0.22      0.20      0.21        20
           2       0.30      0.30      0.30        20

    accuracy                           0.29        66
   macro avg       0.28      0.28      0.28        66
weighted avg       0.28      0.29      0.29        66

Confusion Matrix (Resnet):
[[ 9  9  8]
 [10  4  6]
 [ 9  5  6]]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

auroc_per_class = [roc_auc_score(y_true_classes == i, y_pred_resnet[:, i]) for i in range(len(data))]


for i, auroc in enumerate(auroc_per_class):
    print(f"Class {i} AUROC (Resnet): {auroc}")



Class 0 AUROC (Resnet): 0.4288461538461539
Class 1 AUROC (Resnet): 0.4559782608695652
Class 2 AUROC (Resnet): 0.42010869565217396


In [ ]:
from PIL import Image
image = '/content/drive/MyDrive/HDA2/Covid19-dataset/train/Covid/015.jpg'
image1 = Image.open(image)

In [ ]:
new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)

In [ ]:
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
a = model.predict(x)

1/1 [==============================] - 0s 171ms/step


In [ ]:
new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
x.shape

(1, 224, 224, 3)

In [ ]:
np.argmax(model.predict(x))

1/1 [==============================] - 0s 312ms/step


1

In [ ]:
from PIL import Image
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np

img_path = '/content/drive/MyDrive/HDA2/Covid19-dataset/test/Normal/0103.jpeg'



img = Image.open(img_path)
img = img.resize((224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [ ]:
predictions = model.predict(img_array)


print(predictions)


predicted_class_index = np.argmax(predictions[0])
print("Predicted Class Index:", predicted_class_index)

class_labels = ['class_0', 'class_1', 'class_2']
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class Label:", predicted_class_label)

1/1 [==============================] - 0s 161ms/step
[[1.5023556e-18 1.0000000e+00 2.2782393e-14]]
Predicted Class Index: 1
Predicted Class Label: class_1
